<a href="https://colab.research.google.com/github/Llmortimer/character_morality/blob/main/DissertationFilmAnalyzer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [55]:
!apt-get install -y ffmpeg

import re
import string
import pandas as pd
import numpy as np
import networkx as nx
import plotly.graph_objects as go
import matplotlib.pyplot as plt
import matplotlib.patches as mpl
from difflib import SequenceMatcher
from operator import itemgetter



# title = "IndianaLastCrusade"
# title = "TheAvengers"
# title = "StarWarsTESB"
# title = "HarryPotterDHP2" # not working script issues
# title = "TheDarkKnight"
# title = "TheHungerGames" # script and subtitles
title = "Aladdin"

input_script = '/content/drive/MyDrive/shared/Scripts_and_Films/' + title + '/' + title + 'Script.txt'
input_subtitles = '/content/drive/MyDrive/shared/Scripts_and_Films/' + title + '/' + title + 'Subtitles.srt'
dialogue_script = '/content/drive/MyDrive/shared/Scripts_and_Films/' + title + '/' + title + 'Dialogue.txt'
input_film = '/content/drive/MyDrive/shared/Scripts_and_Films/' + title + '/' + title + 'Film.mp4'
extracted_scenes = '/content/drive/MyDrive/shared/Scripts_and_Films/' + title + '/scenes/time.txt'

Reading package lists... Done
Building dependency tree       
Reading state information... Done
ffmpeg is already the newest version (7:3.4.8-0ubuntu0.2).
0 upgraded, 0 newly installed, 0 to remove and 39 not upgraded.


In [56]:
#pre-process subtitles by extractings start and end times of each dialogue.
f = open(input_subtitles).read()
matches = re.findall(r"([0-9]*)\n(.+)--> (.+)\n((.+\n{0,1})+)", f, re.M)
id = []
start = []
end = []
dialogue_sub = []
for i in matches:
    ex = i[3].rstrip().replace('\n', ' ')
    ex = re.sub('<[^<]+?>', '', ex)
    if title == "HarryPotterDHP2" or title == "TheHungerGames":
        sounds = re.match(r"\([A-Z ]+\)", ex)
    else:
        sounds = re.match(r"\[[A-Z ]+\]", ex)

    names = re.match(r"[A-Z ]+: ", ex)
    if sounds:
        print(sounds)
        continue
    if names:
        print(names.group(0))
        ex = ex.replace(names.group(0), '')

    id.append(i[0])
    start.append(i[1])
    end.append(i[2])
    dialogue_sub.append(ex)

In [57]:
for i in dialogue_sub:
    print(i)

#Oh, I come from a land From a faraway place#
#Where the caravan camels roam#
#Where it's flat and immense And the heat is intense#
#It's barbaric but hey, it's home#
#When the wind's from the east And the sun's from the west#
#And the sand in the glass is right#
#Come on down, stop on by Hop a carpet and fly#
#To another Arabian night#
#Arabian nights#
#Like Arabian days#
#More often than not Are hotter than hot#
#In a lot of good ways#
#Arabian nights#
#'Neath Arabian moons#
#A fool off his guard Could fall and fall hard#
#Out there on the dunes#
Ah, salaam, and good evening to you, worthy friend.
Please, please, come closer.
T oo close. A little too close.
There.
Welcome to Agrabah,
city of mystery,
of enchantment,
and the finest merchandise this side of the river Jordan, on sale today.
Look at this. Yes.
Combination hookah and coffeemaker. Also makes julienne fries.
Will not break. Will not...
It broke.
Ohh! Look at this.
I have never seen one of these intact before.
This is the fa

In [64]:
#pre-process script by extracting the dialogue and character names writing to seperate file.
f = open(input_script).read()
if title == "Aladdin":
    matches = re.findall(r"([A-Z ]+):\s*([A-Za-z ,?.'!\/()\-\s]+)$", f, re.M)
else:
    matches = re.findall(r"\n\s+([A-Z ]+)\n *((.+\s)*)", f, re.M)
dialogue = []
characters = []
for i in matches:
    if '' not in i:
        j = " ".join(i[1].split())
        if title == "aladdin":
            j = re.sub("[\(\[].*?[\)\]]", "", j)
        
        dialogue.append(j)
        characters.append(i[0])

        

with open(dialogue_script, "w") as ds:
    for inst in dialogue:
        ds.write(inst+"\n")

In [53]:
def similar(a, b):
    return SequenceMatcher(None, a, b).ratio()

combined = []
last = 0

for c, i in enumerate(dialogue_sub):
    for count, j in enumerate(dialogue):

        if similar(i, j) > 0.5 and count > last and count < last + 10:
            combined.append((similar(i, j), c, count, i, j))
            last = count
            break

list1 = []
list2 = []

last_sub = -1
last_script = -1

for c in combined:
    if c[1] > last_sub+1:
        list1.append(list(range(last_sub+1, c[1])))
    elif c[1] == last_sub+1:
        list1.append([])

    list1.append([c[1]])
    last_sub = c[1]

    if c[2] > last_script+1:
        list2.append(list(range(last_script+1, c[2])))
    elif c[2] == last_script+1:
        list2.append([])

    list2.append([c[2]])
    last_script = c[2]

list1.append(list(range(last_sub+1, len(dialogue_sub))))
list2.append(list(range(last_script+1, len(dialogue))))

In [54]:
for c in combined:
    print(c)

(0.96, 47, 8, 'We could do it, you know?', 'We could do it, you know.')
(0.6, 67, 11, 'Forty-two.', 'Forget it.')
(0.5333333333333333, 173, 15, "Don't cry.", 'Done.')
(0.5185185185185185, 264, 24, 'There it is.', 'This is stupid.')
(0.5531914893617021, 352, 27, 'One of you will be alive.', 'See you in the square.')
(0.5625, 483, 35, 'Near your head.', 'See for yourself.')
(0.5111111111111111, 494, 41, 'It is the only thing stronger than fear.', 'Don’t want the officials taking us to prison, now.')
(0.5454545454545454, 650, 50, 'You, too?', 'You can’t go!')
(0.5555555555555556, 748, 51, 'Here we go!', 'Let go!')


In [5]:
char_annotated = []

for sbt, sct in zip(list1, list2):
    char = []
    dia = []
    diaSbt = []
    for i in sct:
        char.append(characters[i])
        dia.append(dialogue[i])
    
    for i in sbt:
        diaSbt.append(dialogue_sub[i])

    if len(sbt) != 0 and len(sct) != 0: 
        char_annotated.append([char, sbt, sct, diaSbt, dia, start[sbt[0]], end[sbt[-1]]])
    elif len(sbt) == 0 and len(sct) != 0:
        char_annotated.append([char, sbt, sct, ['no subtitle dialogue'], dia, 'no time info', 'no time info'])
    elif len(sct) == 0 and len(sbt) != 0 :
        char_annotated.append([['null'], sbt, sct, diaSbt, ['no script dialogue'], start[sbt[0]], end[sbt[-1]]])

In [15]:
edits = []

print(len(char_annotated))

for count, dialogue in enumerate(char_annotated):
    #where the speaker of the dialogue is set to 'null'
    if dialogue[0] == ['null']:
        #we only consider the subtitle dialogues to move to other peices of dialogue as monologues exist within the script
        query_dialogues = dialogue[3]
        print(query_dialogues)
        #for every line in the script dialogues
        for i, query_dialogue in enumerate(query_dialogues):
            #check if it is present in the previous or the post dialogues
            if query_dialogue.translate(str.maketrans('', '', string.punctuation)) in char_annotated[count-1][4][0].translate(str.maketrans('', '', string.punctuation)) and query_dialogue not in char_annotated[count-1][3][0]:
                #if its present in the previous script dialogue but not in the subtitles dialogue: add details to a list of edits to do
                edits.append((count, count-1, dialogue[1][i], len(query_dialogues)-i, query_dialogue))
            elif count+1 >= len(char_annotated):
                continue
            elif query_dialogue in char_annotated[count+1][4][0] and query_dialogue not in char_annotated[count+1][3][0]:
                #if it is present in the next script dialogue but not in the subtitles dialogue: add details to a list of edits to do
                edits.append((count, count+1, dialogue[1][i], len(query_dialogues)-i, query_dialogue))


683
['Two guys on the roof. Every guy gets a share. Five shares is plenty.']
['He thinks he can sit it out and still take a slice.']
['And there it goes.', 'Heads down!', "All right, tootsie, you're taking a dive with me.", "- Down! I said, stay down there! - Don't hurt me!", "That's funny, it didn't dial out to 911. It was trying to reach a private number.", '- Is it a problem? - No, I am done here.', 'Sit down! Down!', 'Down! I said, stay down there!', "Obviously we don't want you doing anything with your hands...", '...other than holding on for dear life.', 'On the ground! Stay on the ground!', 'Nobody make a move! Nobody! Stay down!', 'Yeah!', "You have any idea who you're stealing from?", "- You and your friends are dead. - He's out, right?", 'What...?', 'Where did you learn to count?', 'They wired this thing up with like 5000 volts. What kind of bank does that?', "A Mob bank. I guess the Joker's as crazy as they say.", "Where's the alarm guy?", 'Boss told me when the guy was done

In [16]:
for c in edits:
    print(c)

(10, 9, 31, 3, "Where's the alarm guy?")
(43, 42, 79, 2, "I never said they'd be places you wanted to go.")
(68, 67, 111, 2, '...for not sleeping in a mansion.')
(70, 71, 115, 1, 'My armor.')
(98, 97, 155, 2, '...Salvatore Maroni, is the new head of the Falcone crime family.')
(111, 110, 176, 2, 'Fancy stuff for a city cop. Have help?')
(152, 151, 253, 4, 'But this is a democracy, Harvey.')
(210, 209, 420, 1, '...tryouts.')
(224, 225, 446, 1, 'Restricted.')
(238, 237, 470, 3, 'One big pot.')
(292, 291, 578, 12, 'Gotham needs a hero with a face.')
(300, 299, 618, 3, 'What are you doing?')
(336, 335, 708, 2, 'Harvey Dent.')
(357, 356, 790, 1, 'God help whoever he does want to talk to.')
(388, 389, 856, 1, 'Harvey called.')
(399, 400, 872, 2, 'Endure, Master Wayne. Take it.')
(443, 442, 1016, 1, '...I saved him.')
(457, 458, 1047, 1, "I don't want to kill you.")
(471, 472, 1080, 1, 'Look at you go.')
(483, 482, 1110, 3, '...and the freaks like you who would just enjoy it.')
(483, 482, 111

In [17]:
#in the list edits there contains: (index of line in char_annotated to be moved, index of what line it should be in, exact id of the line to be moved, if it is moved forwards or backwards, the line to be moved)
for e in edits:
    if e[0] > e[1]:
        #append the literal line to the previous one 
        char_annotated[e[1]][3].append(e[4])
        #edit the end time of the appended line
        char_annotated[e[1]][6] = end[e[2]]
        #append the id of the line to the previous one
        char_annotated[e[1]][1].append(e[2])
    else:
        #insert the literal line to the beginning previous one 
        char_annotated[e[1]][3].insert(0, e[4])
        #edit the end time of the inserted line
        char_annotated[e[1]][6] = start[e[2]]
        #insert the id of the line to the previous one
        char_annotated[e[1]][1].insert(0, e[2])

i = 0
for e in edits:
    if e[3] == 1:
        del char_annotated[e[0]-i]
        i+=1

In [ ]:
for i in char_annotated:
    print(i)

In [18]:
to_delete = []

for count, i in enumerate(char_annotated):
    if i[0] == ['null']:
        to_delete.append(count)

    if i[5] == 'no time info' and i[6] == 'no time info':
        to_delete.append(count)


for count, i in enumerate(to_delete):
    del char_annotated[i-count]

In [19]:
final_dialogue = []

for i in char_annotated:
    for count, j in enumerate(i[0]):
        temp, ms = i[5].split(',')
        h, m, s = temp.split(':')
        start_time = int(h) * 3600 + int(m) * 60 + int(s) + float(ms)/1000
        temp, ms = i[6].split(',')
        h, m, s = temp.split(':')
        end_time = int(h) * 3600 + int(m) * 60 + int(s) + float(ms)/1000

        final_dialogue.append([j, i[4][count], start_time, end_time])


In [ ]:
for f in final_dialogue:
    print(f)

In [28]:
#scene detection, for each frame assign a probablility that the scene has changed given the change of intensity of the pixels on the pervious frame.
#if this probablility is higher than 0.5 for a given frame the frame time, and probablilty have been output to a file "time.txt" and the frame itself output to a PNG in scenes/ 

!ffmpeg -i /content/drive/MyDrive/shared/Scripts_and_Films/TheDarkKnight/TheDarkKnightFilm.mp4 -filter_complex "select='gt(scene,0.6)',metadata=print:file=/content/drive/MyDrive/shared/Scripts_and_Films/TheDarkKnight/scenes/time.txt" -vsync vfr /content/drive/MyDrive/shared/Scripts_and_Films/TheDarkKnight/scenes/img%03d.png

ffmpeg version 3.4.8-0ubuntu0.2 Copyright (c) 2000-2020 the FFmpeg developers
  built with gcc 7 (Ubuntu 7.5.0-3ubuntu1~18.04)
  configuration: --prefix=/usr --extra-version=0ubuntu0.2 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --enable-gpl --disable-stripping --enable-avresample --enable-avisynth --enable-gnutls --enable-ladspa --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librubberband --enable-librsvg --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvorbis --enable-libvpx --enable-libwavpack --enable-libwebp --enable-libx265 --enable-libxml2 --enable-libxvid --enable-lib

In [29]:
#extract the scene changes and their times from the text file generated by the scene change algorithm

f = open(extracted_scenes).read()
matches = re.findall(r"frame:([0-9]+).+pts_time:([0-9.]+)", f)

scenes = {}
previous = '0'

for match in matches:
    scenes[match[0]] = {}
    scenes[match[0]]['start'] = float(previous)
    scenes[match[0]]['end'] = float(match[1])
    previous = match[1]

In [93]:
print(scenes)

{'0': {'start': 0.0, 'end': 3.9623}, '1': {'start': 3.9623, 'end': 71.7801}, '2': {'start': 71.7801, 'end': 105.063}, '3': {'start': 105.063, 'end': 107.232}, '4': {'start': 107.232, 'end': 171.63}, '5': {'start': 171.63, 'end': 175.968}, '6': {'start': 175.968, 'end': 178.554}, '7': {'start': 178.554, 'end': 184.56}, '8': {'start': 184.56, 'end': 196.363}, '9': {'start': 196.363, 'end': 204.246}, '10': {'start': 204.246, 'end': 207.165}, '11': {'start': 207.165, 'end': 210.335}, '12': {'start': 210.335, 'end': 212.129}, '13': {'start': 212.129, 'end': 214.882}, '14': {'start': 214.882, 'end': 235.485}, '15': {'start': 235.485, 'end': 240.574}, '16': {'start': 240.574, 'end': 246.955}, '17': {'start': 246.955, 'end': 248.582}, '18': {'start': 248.582, 'end': 253.337}, '19': {'start': 253.337, 'end': 255.881}, '20': {'start': 255.881, 'end': 256.757}, '21': {'start': 256.757, 'end': 258.3}, '22': {'start': 258.3, 'end': 258.634}, '23': {'start': 258.634, 'end': 264.056}, '24': {'start':

In [30]:
# eliminate all characters who only speak once
to_eliminate = {}

for c in characters:
    to_eliminate[c] = 0

print(to_eliminate)

for f in final_dialogue:
    to_eliminate[f[0]] += 1

print(to_eliminate)

eliminate = []
for c, i in enumerate(to_eliminate):
    if to_eliminate[i] <= 2:
        eliminate.append(i)

print(eliminate)

to_elim = []
for c in eliminate:
    for i, f in enumerate(final_dialogue):
        if f[0] == c:
            to_elim.append(i)

print(to_elim)

for f in sorted(to_elim, reverse=True):
    del final_dialogue[f]

{'GRUMPY': 0, 'CHUCKLES': 0, 'HAPPY': 0, 'DOPEY': 0, 'BANK MANAGER': 0, 'BOZO': 0, 'THE JOKER': 0, 'DEALER': 0, 'RAMIREZ': 0, 'WUERTZ': 0, 'GORDON': 0, 'CHECHEN': 0, 'JUNKIE': 0, 'SCARECROW': 0, 'BODYGUARD': 0, 'BATMAN': 0, 'ALFRED': 0, 'WAYNE': 0, 'DENT': 0, 'RACHEL': 0, 'ROSSI': 0, 'LAU': 0, 'REESE': 0, 'FOX': 0, 'NATASCHA': 0, 'CRIME BOSS': 0, 'MARONI': 0, 'GAMBOL': 0, 'VP': 0, 'REPORTER': 0, 'JUDGE SURRILLO': 0, 'MAYOR': 0, 'ENGEL': 0, 'VOICE': 0, 'COMMISSIONER LOEB': 0, 'LOEB': 0, 'SURRILLO': 0, 'STEPHENS': 0, 'ASSISTANT DA': 0, 'THUG': 0, 'GENTLEMAN': 0, 'MALE GUEST': 0, 'FEMALE GUEST': 0, 'DRIVER': 0, 'THE MAYOR': 0, 'MAN': 0, 'BARBARA': 0, 'MISTRESS': 0, 'HECKLER': 0, 'ACTING COMISSIONER': 0, 'SHOTGUN SWAT': 0, 'OFFICER': 0, 'RADIO': 0, 'BATMOBILE VOICE': 0, 'PILOT': 0, 'DETECTIVE MURPHY': 0, 'FAT THUG': 0, 'JAMES': 0, 'MEDIC': 0, 'COP': 0, 'SERGEANT': 0, 'UNIFORMED COP': 0, 'NURSE': 0, 'POLK': 0, 'BERG': 0, 'BARTENDER': 0, 'CIVILIAN': 0, 'FIRST MATE': 0, 'NATIONAL GUARD COMMAN

In [31]:
scene_dialogues = []
character_list = []
scene_combined = {}

for s in scenes:
    for d in final_dialogue:
        if d[2] > scenes[s]['start'] and d[3] < scenes[s]['end']:
            # print((s,d))
            scene_dialogues.append((s, d))

for i, j in scene_dialogues:
    scene_combined[i] = []

for i, j in scene_dialogues:
    # if j[0] not in character_list:
    #     character_list.append(j[0])

    scene_combined[i].append(j)

if eliminate:
    character_list = mylist = list(dict.fromkeys([x for x in characters if x not in eliminate]))
else:
    character_list = characters

print(character_list)
# print(scene_combined)
   

['GRUMPY', 'HAPPY', 'BANK MANAGER', 'THE JOKER', 'RAMIREZ', 'GORDON', 'CHECHEN', 'SCARECROW', 'BODYGUARD', 'BATMAN', 'ALFRED', 'WAYNE', 'DENT', 'RACHEL', 'LAU', 'REESE', 'FOX', 'NATASCHA', 'MARONI', 'GAMBOL', 'MAYOR', 'ENGEL', 'VOICE', 'STEPHENS', 'BARBARA', 'SHOTGUN SWAT', 'FAT THUG', 'JAMES', 'BERG', 'BARTENDER', 'SWAT LEADER']


In [ ]:
for s in scene_combined:
    print(scene_combined[s])

In [32]:
graph_matrix = np.zeros((len(character_list), len(character_list)))
current_scene = 0
scene_characters = []
scene_durations = []

for s, d in scene_dialogues:  
    if d[0] not in scene_characters:
        scene_characters.append(d[0])

    #work out a way to calculate weights for each scene individually, sum of total time for each character?

weightings_out = {}
scene_weightings = {}
print(character_list)

for scene in scene_combined:
    for dialogue in scene_combined[scene]:
        if dialogue[0] not in weightings_out:
            weightings_out[dialogue[0]] = 0

        weightings_out[dialogue[0]] = weightings_out[dialogue[0]] + round(d[3] - d[2], 3)

    scene_weightings[scene] = weightings_out
    weightings_out = {}

print(scene_weightings)
longest_dialogue = 0

for i in scene_weightings:
    for j in scene_weightings[i]:
        if scene_weightings[i][j] > longest_dialogue:
            longest_dialogue = scene_weightings[i][j]

['GRUMPY', 'HAPPY', 'BANK MANAGER', 'THE JOKER', 'RAMIREZ', 'GORDON', 'CHECHEN', 'SCARECROW', 'BODYGUARD', 'BATMAN', 'ALFRED', 'WAYNE', 'DENT', 'RACHEL', 'LAU', 'REESE', 'FOX', 'NATASCHA', 'MARONI', 'GAMBOL', 'MAYOR', 'ENGEL', 'VOICE', 'STEPHENS', 'BARBARA', 'SHOTGUN SWAT', 'FAT THUG', 'JAMES', 'BERG', 'BARTENDER', 'SWAT LEADER']
{'10': {'HAPPY': 1.919}, '27': {'GRUMPY': 5.757}, '35': {'BANK MANAGER': 3.838}, '37': {'THE JOKER': 1.919}, '45': {'RAMIREZ': 7.676, 'GORDON': 3.838}, '46': {'CHECHEN': 7.676, 'SCARECROW': 9.595, 'BODYGUARD': 1.919}, '62': {'BATMAN': 3.838}, '64': {'RAMIREZ': 3.838, 'GORDON': 11.514000000000001, 'BATMAN': 5.757}, '71': {'WAYNE': 1.919}, '73': {'WAYNE': 3.838}, '75': {'ALFRED': 1.919}, '78': {'ALFRED': 1.919}, '86': {'DENT': 15.352000000000002, 'RACHEL': 5.757}, '87': {'RACHEL': 5.757}, '88': {'GORDON': 1.919}, '89': {'DENT': 1.919}, '105': {'DENT': 1.919}, '107': {'DENT': 1.919}, '109': {'DENT': 1.919}, '116': {'REESE': 1.919}, '117': {'FOX': 1.919}, '125': {

In [115]:
from networkx.algorithms.community.centrality import girvan_newman
import itertools

comp = girvan_newman(G)
k = 10
for communities in itertools.islice(comp, k):
    print(tuple(sorted(c) for c in communities))


o = tuple(sorted(c) for c in next(comp))
print(o)

(['BEN', 'BOBA FETT', 'CREATURE', 'DECK OFFICER', 'EMPEROR', 'HAN', 'LANDO', 'LEIA', 'LUKE', 'NEEDA', 'OZZEL', 'PIETT', 'RIEEKAN', 'THREEPIO', 'VADER', 'VEERS', 'YODA'], ['ZEV'], ['DACK'], ['WEDGE'])
(['BEN', 'BOBA FETT', 'CREATURE', 'DECK OFFICER', 'HAN', 'LANDO', 'LEIA', 'LUKE', 'NEEDA', 'OZZEL', 'PIETT', 'RIEEKAN', 'THREEPIO', 'VADER', 'VEERS', 'YODA'], ['ZEV'], ['DACK'], ['WEDGE'], ['EMPEROR'])
(['BEN', 'BOBA FETT', 'CREATURE', 'HAN', 'LANDO', 'LEIA', 'LUKE', 'NEEDA', 'OZZEL', 'PIETT', 'RIEEKAN', 'THREEPIO', 'VADER', 'VEERS', 'YODA'], ['DECK OFFICER'], ['ZEV'], ['DACK'], ['WEDGE'], ['EMPEROR'])
(['BEN', 'BOBA FETT', 'CREATURE', 'HAN', 'LANDO', 'LEIA', 'LUKE', 'NEEDA', 'OZZEL', 'THREEPIO', 'VADER', 'YODA'], ['PIETT', 'RIEEKAN', 'VEERS'], ['DECK OFFICER'], ['ZEV'], ['DACK'], ['WEDGE'], ['EMPEROR'])
(['BOBA FETT', 'CREATURE', 'HAN', 'LANDO', 'LEIA', 'LUKE', 'NEEDA', 'OZZEL', 'THREEPIO', 'VADER', 'YODA'], ['PIETT', 'RIEEKAN', 'VEERS'], ['DECK OFFICER'], ['BEN'], ['ZEV'], ['DACK'], ['WE

In [33]:
#collect durations into the graph matrix
idx_in = []
for scene in scene_weightings:
    for char in scene_weightings[scene]:
        idx_out = character_list.index(char)
        for listener in scene_weightings[scene]:
            idx_in.append(character_list.index(listener))

        # print(idx_out)
        # print(idx_in)

        for idx in idx_in:
            graph_matrix[idx_out][idx] += scene_weightings[scene][char]

        idx_in = []

df = pd.DataFrame(graph_matrix)
df = (df).round(3)
df.columns = character_list
df.index = character_list
# print(df)


#generate the graph for degree centrality including all characters
plt.figure(figsize=(20,12))

G = nx.from_pandas_adjacency(df, nx.DiGraph)
weight = nx.get_edge_attributes(G, "weight")
pos = nx.spring_layout(G)
nx.set_node_attributes(G, pos, "pos")

weight_list = [f'{w}' for w in list(nx.get_edge_attributes(G, 'weight').values())]

edge_x = []
edge_y = []
xtext = []
ytext = []
for edge in G.edges():
    x0, y0 = G.nodes[edge[0]]['pos']
    x1, y1 = G.nodes[edge[1]]['pos']
    xtext.append((x0+x1)/2)
    ytext.append((y0+y1)/2)
    edge_x.append(x0)
    edge_x.append(x1)
    edge_x.append(None)
    edge_y.append(y0)
    edge_y.append(y1)
    edge_y.append(None)

# for i in weight:
#     weight_list.append(weight[i])

edge_trace = go.Scatter(
    x=edge_x, y=edge_y,
    line=dict(width=0.5, color='#888'),
    # hoverinfo='none',
    text=weight_list,
    hoverinfo='text',
    mode='lines')


eweights_trace = go.Scatter(x=xtext,y= ytext, mode='text',
    marker_size=0.5,
    text=weight_list,
    textposition='top center',
    hovertemplate='%{text}')


node_x = []
node_y = []
for node in G.nodes():
    x, y = G.nodes[node]['pos']
    node_x.append(x)
    node_y.append(y)

node_trace = go.Scatter(
    x=node_x, y=node_y,
    mode='markers',
    hoverinfo='text',
    marker=dict(
        showscale=True,
        # colorscale options
        #'Greys' | 'YlGnBu' | 'Greens' | 'YlOrRd' | 'Bluered' | 'RdBu' |
        #'Reds' | 'Blues' | 'Picnic' | 'Rainbow' | 'Portland' | 'Jet' |
        #'Hot' | 'Blackbody' | 'Earth' | 'Electric' | 'Viridis' |
        colorscale='Reds',
        # reversescale=True,
        color=[],
        size=10,
        colorbar=dict(
            thickness=15,
            title='Node Connections',
            xanchor='left',
            titleside='right'
        ),
        line_width=2))

node_adjacencies = []
node_text = []
degree_centrality = []
for node, adjacencies in enumerate(G.adjacency()):
    node_adjacencies.append(len(adjacencies[1]))
    node_text.append(character_list[node]+', # of connections: '+str(len(adjacencies[1])))
    degree_centrality.append(len(adjacencies[1]))

# dc = pd.DataFrame(degree_centrality)
node_trace.marker.color = node_adjacencies
node_trace.text = node_text

fig = go.Figure(data=[edge_trace, node_trace],
             layout=go.Layout(
                title='Graph Showing Degree Centrality of Each Node',
                titlefont_size=16,
                showlegend=False,
                hovermode='closest',
                margin=dict(b=20,l=5,r=5,t=40),
                # annotations=[ dict(
                #     text="Python code: <a href='https://plotly.com/ipython-notebooks/network-graphs/'> https://plotly.com/ipython-notebooks/network-graphs/</a>",
                #     showarrow=False,
                #     xref="paper", yref="paper",
                #     x=0.005, y=-0.002 ) ],
                xaxis=dict(showgrid=False, zeroline=False, showticklabels=False),
                yaxis=dict(showgrid=False, zeroline=False, showticklabels=False))
                )
fig.show()

<Figure size 1440x864 with 0 Axes>

In [116]:
import plotly.express as px

fig = px.scatter(x=scene_characters, y=degree_centrality)
fig.update_xaxes(categoryorder='total descending')
fig.show()

In [ ]:
#generate df for network not including indy
adj_mat_noIndy = np.zeros((len(character_list)-1, len(character_list)-1))
idx_in = []
for scene in scene_weightings:
    for char in scene_weightings[scene]:
        if char == 'INDY':
            continue

        idx_out = scene_characters.index(char)
        if idx_out > 8:
            idx_out -= 1

        for listener in scene_weightings[scene]:
            if listener == 'INDY':
                continue
            idx_in_temp = scene_characters.index(listener)
            if idx_in_temp > 8:
                idx_in_temp -= 1

            idx_in.append(idx_in_temp)

        for idx in idx_in:
            adj_mat_noIndy[idx_out][idx] += scene_weightings[scene][char]

        idx_in = []


df_noIndy = pd.DataFrame(adj_mat_noIndy)
df_noIndy = df_noIndy.round(3)
df_noIndy.columns = [x for i,x in enumerate(scene_characters) if x!='INDY']
df_noIndy.index = [x for i,x in enumerate(scene_characters) if x!='INDY']


G = nx.from_pandas_adjacency(df_noIndy, nx.DiGraph)
weight = nx.get_edge_attributes(G, "weight")
pos = nx.spring_layout(G)
nx.set_node_attributes(G, pos, "pos")

edge_x = []
edge_y = []
for edge in G.edges():
    x0, y0 = G.nodes[edge[0]]['pos']
    x1, y1 = G.nodes[edge[1]]['pos']
    edge_x.append(x0)
    edge_x.append(x1)
    edge_x.append(None)
    edge_y.append(y0)
    edge_y.append(y1)
    edge_y.append(None)

weight_list = []
for i in weight:
    weight_list.append(weight[i])

edge_trace = go.Scatter(
    x=edge_x, y=edge_y,
    line=dict(width=0.5, color='#888'),
    hoverinfo='none',
    mode='lines')

node_x = []
node_y = []
for node in G.nodes():
    x, y = G.nodes[node]['pos']
    node_x.append(x)
    node_y.append(y)

node_trace = go.Scatter(
    x=node_x, y=node_y,
    mode='markers',
    hoverinfo='text',
    marker=dict(
        showscale=True,
        # colorscale options
        #'Greys' | 'YlGnBu' | 'Greens' | 'YlOrRd' | 'Bluered' | 'RdBu' |
        #'Reds' | 'Blues' | 'Picnic' | 'Rainbow' | 'Portland' | 'Jet' |
        #'Hot' | 'Blackbody' | 'Earth' | 'Electric' | 'Viridis' |
        colorscale='Reds',
        # reversescale=True,
        color=[],
        size=10,
        colorbar=dict(
            thickness=15,
            title='Node Connections',
            xanchor='left',
            titleside='right'
        ),
        line_width=2))

node_adjacencies = []
node_text = []
for node, adjacencies in enumerate(G.adjacency()):
    node_adjacencies.append(len(adjacencies[1]))
    if node >= 8:
        node+=1
    node_text.append(scene_characters[node]+', # of connections: '+str(len(adjacencies[1])))

node_trace.marker.color = node_adjacencies
node_trace.text = node_text

fig = go.Figure(data=[edge_trace, node_trace],
             layout=go.Layout(
                title='Graph Showing Degree Centrality of Each Node without Indy',
                titlefont_size=16,
                showlegend=False,
                hovermode='closest',
                margin=dict(b=20,l=5,r=5,t=40),
                # annotations=[ dict(
                #     text="Python code: <a href='https://plotly.com/ipython-notebooks/network-graphs/'> https://plotly.com/ipython-notebooks/network-graphs/</a>",
                #     showarrow=False,
                #     xref="paper", yref="paper",
                #     x=0.005, y=-0.002 ) ],
                xaxis=dict(showgrid=False, zeroline=False, showticklabels=False),
                yaxis=dict(showgrid=False, zeroline=False, showticklabels=False))
                )
fig.show()

In [ ]:
from networkx.algorithms.community.centrality import girvan_newman
from networkx.algorithms.community import k_clique_communities
import itertools

comp = girvan_newman(G)
k = 5
for communities in itertools.islice(comp, k):
    print(tuple(sorted(c) for c in communities))


K5 = nx.convert_node_labels_to_integers(G, first_label=2)
G.add_edges_from(K5.edges())
c = list(k_clique_communities(G, 4))

(['OFFICER'], ['FIRST SCOUT'], ['SCOUTMASTER'], ['FEDORA', 'HALFBREED', 'HERMAN', 'MAN', 'PANAMA HAT', 'ROSCOE', 'ROUGH RIDER', 'SHERIFF'], ['BRODY', 'BUTLER', 'DONOVAN', 'ELSA', 'HENRY', 'KNIGHT', 'MAN IN CHAIR', 'NAZI SOLDIER', 'SALLAH', 'SECOND MAN', 'SULTAN', 'VOGEL'], ['IRENE', 'MALE STUDENT'], ['KAZIM'], ['WOMAN'])
(['OFFICER'], ['FIRST SCOUT'], ['SCOUTMASTER'], ['FEDORA', 'HALFBREED', 'HERMAN', 'MAN', 'PANAMA HAT', 'ROSCOE', 'ROUGH RIDER', 'SHERIFF'], ['BRODY', 'BUTLER', 'DONOVAN', 'ELSA', 'HENRY', 'KNIGHT', 'MAN IN CHAIR', 'NAZI SOLDIER', 'SALLAH', 'SECOND MAN', 'VOGEL'], ['IRENE', 'MALE STUDENT'], ['KAZIM'], ['WOMAN'], ['SULTAN'])
(['OFFICER'], ['FIRST SCOUT'], ['SCOUTMASTER'], ['FEDORA', 'HALFBREED', 'HERMAN', 'MAN', 'PANAMA HAT', 'ROSCOE', 'ROUGH RIDER', 'SHERIFF'], ['BRODY', 'DONOVAN', 'ELSA', 'HENRY', 'KNIGHT', 'MAN IN CHAIR', 'NAZI SOLDIER', 'SALLAH', 'SECOND MAN', 'VOGEL'], ['IRENE', 'MALE STUDENT'], ['KAZIM'], ['BUTLER'], ['WOMAN'], ['SULTAN'])
(['OFFICER'], ['FIRST SCO

NetworkXNotImplemented: ignored